In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:11 

In [ ]:
!pip install html_table_extractor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
def table_to_markdown(table):
    """
    Chuyển đổi phần tử bảng HTML thành văn bản Markdown.

    Parameters:
        table (bs4.element.Tag): Phần tử bảng HTML.

    Returns:
        str: Văn bản Markdown của bảng.
    """
    markdown_table = ""
    # Số cột của bảng
    num_columns = len(table.find_all('th'))

    # Header của bảng
    markdown_table += "|" + "|".join([th.get_text().strip() for th in table.find_all('th')]) + "|\n"
    markdown_table += "|-" + "-|-".join(["" for _ in range(num_columns)]) + "|\n"

    # Dữ liệu của bảng
    for row in table.find_all('tr')[1:]:
        markdown_table += "|" + "|".join([td.get_text().strip() for td in row.find_all('td')]) + "|\n"
    return markdown_table

def get_txt_from_dvc(url, file_path):
    """
    Function getting TXT files used in Model Langchain.

    Input: url of Public Sector
    Output: A string of processed html
            File .txt
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    # Xem chi tiết
    driver.find_element("xpath","//a[text()='Xem chi tiết']").click()
    time.sleep(1)

    # Lấy html
    div_element = driver.find_element("xpath","//div[@class='modal-body']")
    div_html = div_element.get_attribute('innerHTML')

    # Sử dụng BeautifulSoup để phân tích HTML
    soup = BeautifulSoup(div_html, 'html.parser')
    html_tables = []
    final_str = ''
    result = ''
    duplicated = []
    link_spans = soup.find_all('span', class_='link')

    for span in link_spans:
          # Lấy ra mã từ thuộc tính onclick
          onclick_value = span.get('onclick')
          start_index = onclick_value.find("(") + 1
          end_index = onclick_value.find(")")
          maudon_id = onclick_value[start_index:end_index]

          # Nối mã vào tên file
          span.string = f'<a href="https://csdl.dichvucong.gov.vn/web/jsp/download_file.jsp?ma={maudon_id.strip()}">{span.string}</a>'

    # Nếu phần tử này được tìm thấy
    if soup:
        added_content = []  # Danh sách để lưu trữ các nội dung đã thêm vào kết quả
        added_tables = []   # Danh sách để lưu trữ các bảng đã thêm vào kết quả
        for element in soup.descendants:
            if element is not None:
                if element.name == 'table':
                    # Kiểm tra xem bảng đã được thêm vào kết quả chưa
                    if element not in added_tables:
                        # Chuyển đổi HTML của bảng thành văn bản Markdown
                        markdown_table = table_to_markdown(element)
                        # Thêm văn bản Markdown của bảng vào kết quả
                        result += markdown_table + '\n'
                        # Thêm bảng vào danh sách đã thêm
                        added_tables.append(element)
                else:
                    if element.find_parent('table') is None:
                      if element.string and str(element.string).strip():
                        result += str(element.string).strip() + '\n'
                        # Thêm nội dung vào danh sách đã thêm
                        added_content.append(str(element.string).strip())

    sentences = result.split('\n')
    unique_sentences = []
    for i, sentence in enumerate(sentences):
        if i == 0 or sentence != sentences[i-1]:
            unique_sentences.append(sentence)
    unique_result = '\n'.join(unique_sentences)
    to_remove = "Doan nay da duoc fix"

    unique_result = unique_result.replace(to_remove, "")

    with open(file_path, "w", encoding="utf-8") as file:
        file.write(unique_result)

    driver.quit()
    return unique_result

link_list = []

# Mở tập tin links.txt để đọc (mặc định mode='r' cho đọc)
with open("/content/link_faq_dvc.txt", 'r') as file:
    # Đọc từng dòng của tập tin và thêm vào danh sách
    link_list = file.readlines()

In [ ]:
seen = set()
uniq = []
for x in link_list:
    if x not in seen:
        uniq.append(x)
        seen.add(x)

In [ ]:
for i in range(0, len(uniq)):
    url = uniq[i][:-1]
    name = url.split('=')[1]
    file_path = f"/content/drive/MyDrive/{name}.txt"
    x = get_txt_from_dvc(url,file_path)

Cào link FAQ:

In [ ]:
def link_retriever(n, file_path):
    """
    A function for retriever a .txt of links based on provided number

    Input: n (the number of pages - each page has 10 link)
           file_path: path to save
    Output: a list of link
            .txt file
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get('https://dichvucong.gov.vn/p/home/dvc-cau-hoi-pho-bien.html?pObject=1#tatca')
    driver.find_element("xpath","//ul[@class='tabs -style01']/li/a").click()
    desired_page_number = n
    page_number = "//div[@class='pagination-bottom']/div[2]//li[@jp-data = '{}']"

    link_dvc = []
    dvc_container = driver.find_elements("xpath","//div[@class='tab-content']/div[1]/div[1]//a")
    for link_container in dvc_container:
        link = link_container.get_attribute("href")
        link_dvc.append(link)

    for i in range(2, n):
        container = driver.find_element("xpath",page_number.format(i)).click()
        time.sleep(2)

        dvc_container = driver.find_elements("xpath","//div[@class='tab-content']/div[1]/div[1]//a")
        for link_container in dvc_container:
            link = link_container.get_attribute("href")
            link_dvc.append(link)

    file_name = file_path
    with open(file_name, "w") as file:
        for link in link_dvc:
            file.write(link + "\n")
    return link_dvc

link_path = "/content/faqs.txt"
link_faq = link_retriever(947, link_path)

In [ ]:
def link_faq_retriever(link_list):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)

    link_dvc = []
    df_faq = pd.DataFrame(columns=['Question', 'Answer', 'Docs'])
    with open('link_faq_dvc.txt', 'a') as file:
        for i in link_list:
            driver.get(i)

            try:
                question = driver.find_element("xpath", "//h1[@class='main-title-sub']").text
            except NoSuchElementException:
                question = None

            try:
                answer = driver.find_element("xpath", "//div[@class='article']/p[3]").text
            except NoSuchElementException:
                answer = None

            # if question and answer:
            #     df_faq = pd.concat([df_faq, pd.DataFrame({'Question': [question], 'Answer': [answer], 'Docs' : []})], ignore_index=True)

            try:
                dvc_container = driver.find_elements("xpath", "//ul[@class='list-document']//a")
                link_docs = []
                for link_container in dvc_container:
                    link = link_container.get_attribute("href")
                    link_dvc.append(link)
                    link_docs.append(link)
                    file.write(link + '\n')
                    print(f"Written link to file: {link}")
                if question and answer:
                    df_faq = pd.concat([df_faq, pd.DataFrame({'Question': [question], 'Answer': [answer], 'Docs' : [link_docs]})], ignore_index=True)
            except NoSuchElementException:
                pass

    driver.quit()
    return link_dvc, df_faq

In [ ]:
file_path = '/content/faqs.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()
link_list = [line.strip() for line in lines][:1000]

In [ ]:
link_new_dvc, faq_list = link_faq_retriever(link_list)

Written link to file: https://dichvucong.gov.vn/p/home/dvc-chi-tiet-thu-tuc-hanh-chinh.html?ma_thu_tuc=1.003755
Written link to file: https://dichvucong.gov.vn/p/home/dvc-chi-tiet-thu-tuc-hanh-chinh.html?ma_thu_tuc=1.005003
Written link to file: https://dichvucong.gov.vn/p/home/dvc-chi-tiet-thu-tuc-hanh-chinh.html?ma_thu_tuc=1.005064
Written link to file: https://dichvucong.gov.vn/p/home/dvc-chi-tiet-thu-tuc-hanh-chinh.html?ma_thu_tuc=1.000994
Written link to file: https://dichvucong.gov.vn/p/home/dvc-chi-tiet-thu-tuc-hanh-chinh.html?ma_thu_tuc=1.003904
Written link to file: https://dichvucong.gov.vn/p/home/dvc-chi-tiet-thu-tuc-hanh-chinh.html?ma_thu_tuc=1.000994
Written link to file: https://dichvucong.gov.vn/p/home/dvc-chi-tiet-thu-tuc-hanh-chinh.html?ma_thu_tuc=1.004697
Written link to file: https://dichvucong.gov.vn/p/home/dvc-chi-tiet-thu-tuc-hanh-chinh.html?ma_thu_tuc=1.003770
Written link to file: https://dichvucong.gov.vn/p/home/dvc-chi-tiet-thu-tuc-hanh-chinh.html?ma_thu_tuc=1

In [ ]:
filtered_faq_list.to_csv('faqs_new.csv',index=False)

In [ ]:
len(filtered_faq_list)

747

In [ ]:
filtered_faq_list = faq_list[faq_list['Docs'].map(lambda x: len(x) > 0)]